In [1]:
import os
import time
import matplotlib.pyplot as plt
import gc; gc.enable() # memory is tight
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from tensorflow import keras

from skimage.io import imread
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from typing import List
from google.cloud import bigquery, storage
from google.cloud import aiplatform as vertex_ai

import tensorflow as tf
from google_cloud_pipeline_components.experimental.custom_job import utils
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component

In [2]:
PROJECT = 'mle-airbus-detection-smu' # Change to your project id.
PROJECT_NUMBER = '484894607141'
REGION = 'asia-east1' # Change to your region.
BUCKET = 'mle_airbus_dataset' # Change to your bucket name.
SERVICE_ACCOUNT = "service-account-for-mle@mle-airbus-detection-smu.iam.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
!gcloud config set project {PROJECT}
    
    
PARENT = f"projects/{PROJECT}/locations/{REGION}"

PIPELINE_NAME = 'airbusmlepipeline'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/{}/pipeline_root'.format(
    BUCKET, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/{}/pipeline_module'.format(
    BUCKET, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/{}/data'.format(BUCKET, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/{}/serving_model'.format(
    BUCKET, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('MODULE_ROOT: {}'.format(MODULE_ROOT))
print('DATA_ROOT: {}'.format(DATA_ROOT))
print('SERVING_MODEL_DIR: {}'.format(SERVING_MODEL_DIR))

BQ_DATASET_NAME = 'mle-airbus-detection-smu.airbus_label_dataset' # Change to your BQ dataset name.
BQ_TABLE_NAME = 'airbus_label'
BQ_LOCATION = ' asia-east1'
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)
print("Vertex API Parent URI:", PARENT)

! gsutil ls -al "gs://"$BUCKET


storage_client = storage.Client(project=PROJECT)
bucket = storage_client.get_bucket(BUCKET)
storage_path = f"gs://{BUCKET}/train_v2/"
print (storage_path)

VERSION = 'v01'
DATASET_DISPLAY_NAME = 'airbus-ship-dataset-display'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'

WORKSPACE = f'gs://{BUCKET}/{DATASET_DISPLAY_NAME}'
EXPERIMENT_ARTIFACTS_DIR = os.path.join(WORKSPACE, 'experiments')

TENSORBOARD_DISPLAY_NAME = f'tb-{DATASET_DISPLAY_NAME}'
EXPERIMENT_NAME = f'{MODEL_DISPLAY_NAME}'

BATCH_SIZE = 16
EDGE_CROP = 16
NB_EPOCHS = 5
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (1, 1)
# number of validation images to use
VALID_IMG_COUNT = 400
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False
N_SAMPLE = 10
IMG_SHAPE = (768, 768)

Updated property [core/project].


PIPELINE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_root
MODULE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_module
DATA_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/data
SERVING_MODEL_DIR: gs://mle_airbus_dataset/airbusmlepipeline/serving_model
Project ID: mle-airbus-detection-smu
Region: asia-east1
Bucket name: mle_airbus_dataset
Service Account: service-account-for-mle@mle-airbus-detection-smu.iam.gserviceaccount.com
Vertex API Parent URI: projects/mle-airbus-detection-smu/locations/asia-east1
gs://mle_airbus_dataset/train_v2/


In [3]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].strip()
    print("Service Account:", SERVICE_ACCOUNT)

In [4]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator "gs://"$BUCKET

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer "gs://"$BUCKET

                                 gs://mle_airbus_dataset/train_v2/


No changes made to gs://mle_airbus_dataset/


# Setup Infra on GCP

In [5]:
import json
import os
import sys

import numpy as np
from google.cloud import aiplatform, bigquery
from google.cloud.aiplatform import gapic as aip

aiplatform.init(project=PROJECT, location=REGION, staging_bucket=BUCKET)

In [6]:
TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

TRAIN_VERSION = "tf-gpu.2-8"
DEPLOY_VERSION = "tf2-gpu.2-8"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

Training: us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest AcceleratorType.NVIDIA_TESLA_K80 1
Deployment: us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest AcceleratorType.NVIDIA_TESLA_K80 1
Train machine type n1-standard-4
Deploy machine type n1-standard-4


In [7]:
QUERY = "SELECT * FROM `mle-airbus-detection-smu.airbus_label_dataset.train`"

BQ_SOURCE_TRAIN = "bq://mle-airbus-detection-smu.airbus_label_dataset.train"
BQ_SOURCE_TEST = "bq://mle-airbus-detection-smu.airbus_label_dataset.test"

In [8]:
bqclient = bigquery.Client()

# Download a table.
table = bigquery.TableReference.from_string(
    "mle-airbus-detection-smu.airbus_label_dataset.train"
)
rows = bqclient.list_rows(
    table
)
dataframe = rows.to_dataframe(
    # Optionally, explicitly request to use the BigQuery Storage API. As of
    # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
    # API is used by default.
    create_bqstorage_client=True,
)

No changes made to gs://mle_airbus_dataset/


         ImageId                                      EncodedPixels  ships  \
0  000155de5.jpg  264661 17 265429 33 266197 33 266965 33 267733...      1   
1  00031f145.jpg  340363 1 341129 4 341896 6 342663 7 343429 10 ...      1   
2  000d42241.jpg  369226 3 369992 5 370760 5 371528 5 372296 5 3...      1   
3  000e6378b.jpg  224667 2 225431 6 226196 10 226962 12 227730 1...      1   
4  001234638.jpg  131064 1 131831 3 132599 5 133366 7 134133 9 1...      1   

   has_ship  
0       1.0  
1       1.0  
2       1.0  
3       1.0  
4       1.0  


In [9]:
dataframe

,ImageId,EncodedPixels,ships,has_ship
0,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,1,1.0
1,00031f145.jpg,340363 1 341129 4 341896 6 342663 7 343429 10 ...,1,1.0
2,000d42241.jpg,369226 3 369992 5 370760 5 371528 5 372296 5 3...,1,1.0
3,000e6378b.jpg,224667 2 225431 6 226196 10 226962 12 227730 1...,1,1.0
4,001234638.jpg,131064 1 131831 3 132599 5 133366 7 134133 9 1...,1,1.0
...,...,...,...,...
148299,ff7ac7f3c.jpg,452546 1 453313 3 454079 5 454846 7 455613 9 4...,15,1.0
148300,ff7ac7f3c.jpg,73142 1 73909 4 74676 6 75443 8 76212 9 76981 ...,15,1.0
148301,ff7ac7f3c.jpg,357792 2 358558 4 359324 7 360092 7 360861 6 3...,15,1.0
148302,ff7ac7f3c.jpg,327886 1 328652 4 329419 6 330188 5 330957 5 3...,15,1.0
